In [1]:
! pip install geopandas pandas sqlalchemy psycopg2-binary openpyxl geoalchemy2 python-dotenv

/bin/bash: /media/mutakabbir/HDD_2TB_02/Forest_Fire/.venv/bin/pip: /media/mutakabbir/HDD_2TB_01/Forest_Fire/.venv/bin/python: bad interpreter: No such file or directory


In [2]:
import geopandas as gpd
from sqlalchemy import create_engine

import os
from dotenv import load_dotenv

In [3]:
CBF_SHAPEFILE_PATH = "../../data/canada/lfed000b21a_e.zip"
PATH_TO_DOT_ENV = "../.env"

DATABASE_TYPE = "postgresql"
DATABASE_HOST = "localhost"

CANADIAN_SUBDIVISON_TABLE_NAME = "S_c"
CANADIAN_BOUNDARY_TABLE_NAME = "C"

In [4]:
load_dotenv(PATH_TO_DOT_ENV)

DATABASE_NAME = os.environ.get("DATABASE_NAME")
POSTGRES_USER = os.environ.get("POSTGRES_USER")
POSTGRES_PASSWORD = os.environ.get("POSTGRES_PASSWORD")
POSTGRES_HOST_PORT = os.environ.get("POSTGRES_HOST_PORT")
POSTGRES_CONTAINER_PORT = os.environ.get("POSTGRES_CONTAINER_PORT")

In [5]:
engine = create_engine(f"{DATABASE_TYPE}://{POSTGRES_USER}:{POSTGRES_PASSWORD}@{DATABASE_HOST}:{POSTGRES_HOST_PORT}/{DATABASE_NAME}")

In [6]:
query_geometry = f"""select sc.geometry from "{CANADIAN_SUBDIVISON_TABLE_NAME}" sc"""

In [7]:
geometry_gdf = gpd.read_postgis(
    sql=query_geometry, 
    con=engine, 
    geom_col="geometry", 
    crs="EPSG:4326"
    )

In [8]:
%%time
canada_gdf = geometry_gdf.dissolve()

CPU times: user 2min 15s, sys: 2.75 s, total: 2min 18s
Wall time: 2min 18s


In [9]:
canada_gdf.to_postgis(
    name=CANADIAN_BOUNDARY_TABLE_NAME,
    con=engine, 
    if_exists='replace'
)  